In [67]:
# импорт библиотек
import numpy as np
import pandas as pd 
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
from sklearn import datasets
iris = datasets.load_iris()

## 3 Функции потерь и оптимизация

### Загрузка данных и преобразование в `DataFrame`
* оставляем только `Versicolor`, `Iris Virginica`
* разделяем та тестовую и тренировочную выборку `20 / 80`

In [68]:
# Загружаем датасет с ирисами
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Фильтруем данные, оставляя только два класса: Versicolor и Virginica
df = df[df['target'].isin([1, 2])]
# Переименовываем колонки для удобства
df = df.rename(columns={
    'sepal length (cm)': 'sepal_length',
    'sepal width (cm)': 'sepal_width',
    'petal length (cm)': 'petal_length',
    'petal width (cm)': 'petal_width'
})
# Получаем данные для признаков (X) и целевой переменной (y)
df_X, df_y = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], df['target']

### `LogisticRegressionWithOutLib` написанная без использования библиотеки
* Без оптимизации
* С RMSProp
* С Nadam
* оценка: $accuracy\ score=\frac{Общее\ количество\ предсказаний}{Количество\  правильных\  предсказаний}$

In [71]:
from sklearn.metrics import accuracy_score # доля правильных 
import time 
from sklearn. model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)
X_train.shape , y_train.shape

class LogisticRegressionWithOutLib:
    def __init__(self, learning_rate, epoch, optimizer=None):
        """
        Инициализация логистической регрессии.
        Параметры:
        - learning_rate (float): Скорость обучения (learning rate).
        - epoch (int): Количество итераций градиентного спуска.
        - optimizer (str): Метод оптимизации: None (без оптимизации), 'RMSProp' или 'Nadam'.
        """
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.optimizer = optimizer
        self.weights = None
        self.bias = None
        self.gradient_sq = None
        self.mu = None

        # параметры RMSProp
        self.beta = 0.9
        self.epsilon = 1e-7

        # параметры Nadam
        self.beta1 = 0.9
        self.beta2 = 0.999

    def sigmoid(self, z):
        # Сигмоидная функция активации
        return 1 / (1 + np.exp(-z))

    def initialize_parameters(self, num_features):
        # Инициализация весов и смещения модели
        self.weights = np.zeros(num_features)
        self.bias = 0
        self.gradient_sq = np.zeros(num_features)
        self.mu = np.zeros(num_features)

    def compute_cost(self, y, y_pred):
        # избежать деления на ноль
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def rmsprop_update(self, dw):
        # Обновление параметров с использованием метода RMSProp
        self.gradient_sq = self.beta * self.gradient_sq + (1 - self.beta) * dw**2
        self.weights -= (self.learning_rate / (np.sqrt(self.gradient_sq) + self.epsilon)) * dw

    def nadam_update(self, dw):
    # Обновление параметров с использованием метода Nadam
        self.mu = self.beta1 * self.mu + (1 - self.beta1) * dw
        self.gradient_sq = self.beta2 * self.gradient_sq + (1 - self.beta2) * dw**2
        mu_hat = self.mu / (1 - self.beta1)
        grad_hat = dw / (1 - self.beta2)
        self.weights -= self.learning_rate * (mu_hat / (np.sqrt(self.gradient_sq) + self.epsilon) + self.beta1 * grad_hat / (1 - self.beta1))

    def gradient_descent(self, X, y):
        num_samples, num_features = X.shape
        self.initialize_parameters(num_features)

        for _ in range(self.epoch):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(linear_model)

            # Заменяем значения y_pred, чтобы избежать точного совпадения с 0 или 1
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

            # для наглядности и поиска ошибок
            cost = self.compute_cost(y, y_pred)

            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            if self.optimizer == 'RMSProp':
                self.rmsprop_update(dw)
            elif self.optimizer == 'Nadam':
                self.nadam_update(dw)
            else:
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            # Дополнительно выводим значение стоимости на каждой итерации
            # if _ % 100 == 0:
            #     print(f"Iteration {_}: Cost {cost}")

    def fit(self, X, y):
        # Обучение модели
        self.gradient_descent(X, y)

    def predict(self, X):
        # Предсказание классов для новых данных
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.sigmoid(linear_model)
        y_pred_cls = [1 if i > 0.5 else 0 for i in y_pred]
        return y_pred_cls

def fit_different_method(
        X_train, X_test, y_train, y_test,
        learning_rate, epoch
        ):
    # проходим циклом по всем трем вариантам с разными оптимизациями и без
    results_df = None
    for model, method_name in [
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer=None),
            "Без оптимизации"),
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer='RMSProp'),
            "С RMSProp"),
        (LogisticRegressionWithOutLib(
            learning_rate=learning_rate,
            epoch=epoch,
            optimizer='Nadam'),
            "С Nadam")]:

        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        end_time = time.time()
        execution_time = end_time - start_time

        result_df = pd.DataFrame([
            [method_name, accuracy, execution_time, learning_rate, epoch]],
            columns=["Метод", "Метрика", "Время работы", "learning_rate", "epoch"])
        results_df = pd.concat([results_df, result_df], ignore_index=True)

    return results_df

results_df = fit_different_method(
    X_train, X_test, y_train, y_test, 0.1, 1000)
results_df

,Метод,Метрика,Время работы,learning_rate,epoch
0,Без оптимизации,0.6,0.446548,0.1,1000
1,С RMSProp,0.6,0.446828,0.1,1000
2,С Nadam,0.6,0.444133,0.1,1000


### Сравнение результата с моделью `sklearn`

In [72]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
test_accuracy = logistic_regression.score(X_test, y_test)
print("Точность на тестовом наборе:", test_accuracy)


Точность на тестовом наборе: 0.95


### Вывод
Исходя из результатов, можно сделать следующие выводы:
* методы оптимизации `RMSProp` и `Nadam` также не приводят к значительному улучшению метрики качества (`accuracy`) на тестовой выборке по сравнению с моделью без оптимизации.
* Метрика качества для всех трех методов составляет `0.6`
* Вероятно задача классификации ирисов хорошо решается простой логистической регрессией 😄 и не требует методов оптимизации
* при этом логистическая модель из библиотеки `sklearn` уверенно показала высокий результат